In [1]:
import pandas as pd
X_train = pd.read_csv('train_tit.csv')
X_test = pd.read_csv('test_tit.csv')

In [3]:
X_train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [4]:
X_test.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


Replace categorical features with number features

In [5]:
X_tr = X_train[['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
X_tr.Sex.replace(to_replace=dict(female=0, male=1), inplace=True)

C:\FolderForAnaconda\envs\ipykernel_py3\lib\site-packages\pandas\core\generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [6]:
X_tr.head(3)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,3,1,22.0,1,0,7.2500
1,2,1,0,38.0,1,0,71.2833
2,3,3,0,26.0,0,0,7.9250


In [7]:
X_te = X_test[['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
X_te.Sex.replace(to_replace=dict(female=0, male=1), inplace=True)

In [8]:
X_te.head(3)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,3,1,34.5,0,0,7.8292
1,893,3,0,47.0,1,0,7.0000
2,894,2,1,62.0,0,0,9.6875


Replace "NaN" values with mean value of this category

In [9]:
X_tr_new = X_tr.fillna(X_tr.mean())
X_tr_new.head(3)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,3,1,22.0,1,0,7.2500
1,2,1,0,38.0,1,0,71.2833
2,3,3,0,26.0,0,0,7.9250


In [10]:
y_train = X_train['Survived']

Predictions based on row data with RF

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
parameter_grid = {
            'criterion': ['entropy', 'gini'],
            'max_depth': [10, 20, 100],
            'n_estimators': [10, 20, 100]
        }
clf = RandomForestClassifier()
grid_searcher = GridSearchCV(clf, parameter_grid, verbose=2)
grid_searcher.fit(X_tr_new, y_train)
clf_best = grid_searcher.best_estimator_
 
print('Best params = ', clf_best.get_params())

C:\FolderForAnaconda\envs\ipykernel_py3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] criterion=entropy, max_depth=10, n_estimators=10 ................
[CV] . criterion=entropy, max_depth=10, n_estimators=10, total=   0.1s
[CV] criterion=entropy, max_depth=10, n_estimators=10 ................
[CV] . criterion=entropy, max_depth=10, n_estimators=10, total=   0.0s
[CV] criterion=entropy, max_depth=10, n_estimators=10 ................
[CV] . criterion=entropy, max_depth=10, n_estimators=10, total=   0.0s
[CV] criterion=entropy, max_depth=10, n_estimators=20 ................
[CV] . criterion=entropy, max_depth=10, n_estimators=20, total=   0.1s
[CV] criterion=entropy, max_depth=10, n_estimators=20 ................
[CV] . criterion=entropy, max_depth=10, n_estimators=20, total=   0.1s
[CV] criterion=entropy, max_depth=10, n_estimators=20 ................
[CV] . criterion=entropy, max_depth=10, n_estimators=20, total=   0.1s
[CV] criterion=entropy, max_depth=10, n_estimators=100 ...............
[CV]  criterion=

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:    8.0s finished


Best params =  {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [13]:
clf_rf = RandomForestClassifier(n_jobs= 1, n_estimators=100, criterion ='entropy', min_impurity_split= None, max_depth=10)

In [14]:
clf_rf.fit(X_tr_new, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [17]:
predicted_rf = clf_rf.predict(X_te_new)

In [18]:
 d_rf = {"PassengerId":X_te_new['PassengerId'], "Survived": pd.Series(predicted_rf)}

In [20]:
df2 = pd.DataFrame(d_rf)
df2.head(3)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0


In [24]:
df1.to_csv('kaggle_note_2', sep=',', index=False)

Improved version

In [29]:
import pandas as pd
X_train = pd.read_csv('train_tit.csv')
X_test = pd.read_csv('test_tit.csv')

In [30]:
X_tr = X_train[['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
X_tr.Sex.replace(to_replace=dict(female=0, male=1), inplace=True)

In [31]:
X_te = X_test[['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
X_te.Sex.replace(to_replace=dict(female=0, male=1), inplace=True)

In [32]:
y_train = X_train['Survived']

Fill 'Nan' age values with mean age value of men and women

In [33]:
female_mean = (X_tr['Sex']==0).mean()
fe_mean = female_mean*100
int(fe_mean) #female mean age

35

In [34]:
male_mean = (X_tr['Sex']==1).mean()
ma_mean = male_mean*100
int(ma_mean) # male mean age

64

In [35]:
mask =  pd.isnull(X_tr['Age']) & X_tr.Sex.eq(0)
X_tr.loc[mask, ['Age']] = int(fe_mean)

In [36]:
mask_1 =  pd.isnull(X_tr['Age']) & X_tr.Sex.eq(1)
X_tr.loc[mask_1, ['Age']] = int(ma_mean)

In [37]:
mask_test =  pd.isnull(X_te['Age']) & X_te.Sex.eq(0)
X_te.loc[mask_test, ['Age']] = int(fe_mean)

In [38]:
mask_1_test =  pd.isnull(X_te['Age']) & X_te.Sex.eq(1)
X_te.loc[mask_1_test, ['Age']] = int(ma_mean)

The same with 'Nan' Fare values

In [39]:
fare_test = (X_te['Fare']).mean()
fare_test
mask_3_test =  pd.isnull(X_te['Fare']) 
X_te.loc[mask_3_test, ['Fare']] = fare_test

In [40]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [41]:
X = X_tr[X_tr.columns[1:8]]

In [42]:
from sklearn import preprocessing
X_scaled = preprocessing.scale(X)

Use cross-validation to find optimal k in KNN algorithm

In [44]:
from sklearn.neighbors import KNeighborsClassifier
array=[]
results=[]
k=1
while k<51:
    neigh = KNeighborsClassifier(n_neighbors=k)
    results = cross_val_score(neigh, X_scaled, y_train, cv=kf, scoring='accuracy')
   # print results.mean()
    array.append(results.mean())
    k = k+1
max_res = max(array)
print (max_res)
array.index(max_res)+1

0.8103571652752496


18

In [45]:
neigh = KNeighborsClassifier(n_neighbors=18)

In [46]:
neigh.fit(X_scaled, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=18, p=2,
                     weights='uniform')

In [47]:
X_te_new = X_te[X_te.columns[1:7]]
X_te_new.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,34.5,0,0,7.8292
1,3,0,47.0,1,0,7.0000
2,2,1,62.0,0,0,9.6875
3,3,1,27.0,0,0,8.6625
4,3,0,22.0,1,1,12.2875


In [48]:
X.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,22.0,1,0,7.2500
1,1,0,38.0,1,0,71.2833
2,3,0,26.0,0,0,7.9250
3,1,0,35.0,1,0,53.1000
4,3,1,35.0,0,0,8.0500


In [49]:
X_test_scaled = preprocessing.scale(X_te_new)

In [50]:
pred = neigh.predict(X_test_scaled)

In [51]:
d_knn = {"PassengerId":X_te['PassengerId'], "Survived": pd.Series(pred)}
df5 = pd.DataFrame(d_knn)


In [53]:
df5.to_csv('kaggle_note_5', sep=',', index=False) 

# score: 0.75598

Test this data with logistic regression

In [55]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_scaled, y_train)

In [56]:
prediction = clf.predict(X_test_scaled)

In [57]:
d_logr = {"PassengerId":X_te['PassengerId'], "Survived": pd.Series(prediction)}
df6 = pd.DataFrame(d_logr)
df6.to_csv('kaggle_note_6', sep=',', index=False)

#score: 0.75598 - the same 

Let's replace categorical Embarked fatures with numbers

In [58]:
X_new = X.join( X_train['Embarked'])
X_new.Embarked.replace(to_replace=dict(C=0, Q=1, S=2), inplace=True)
X_new = X_new.fillna(2)

In [59]:
from sklearn import preprocessing
X_scaled = preprocessing.scale(X_new)

In [61]:
from sklearn.neighbors import KNeighborsClassifier
array=[]
results=[]
k=1
while k<51:
    neigh = KNeighborsClassifier(n_neighbors=k)
    results = cross_val_score(neigh, X_scaled, y_train, cv=kf, scoring='accuracy')
   # print results.mean()
    array.append(results.mean())
    k = k+1
max_res = max(array)
print (max_res)
array.index(max_res)+1

0.8238089259933463


15

In [62]:
neigh1 = KNeighborsClassifier(n_neighbors=15)

In [63]:
neigh1.fit(X_scaled, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                     weights='uniform')

In [64]:
X_test_new = X_te_new.join( X_test['Embarked'])
X_test_new.Embarked.replace(to_replace=dict(C=0, Q=1, S=2), inplace=True)
X_test_scaled_new = preprocessing.scale(X_test_new)

In [65]:
pr = neigh1.predict(X_test_scaled_new)

In [66]:
d_logr_2 = {"PassengerId":X_te['PassengerId'], "Survived": pd.Series(pr)}
df7 = pd.DataFrame(d_logr_2)
df7.to_csv('kaggle_note_7', sep=',', index=False)

#score:0.75598 - not improved

Let's take into consideration the Cabin and Deck and try to make predictions using this data

In [67]:
X_new = X_new.join( X_train['Cabin'])

In [70]:
import string
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if big_string.find(substring) != -1:
            return substring
    return np.nan

In [71]:
import numpy as np
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
X_new['Deck']=X_new['Cabin'].map(lambda x: substrings_in_string(str(x), cabin_list))

In [72]:
del X_new['Cabin']

In [73]:
def make_conj(data, f1, f2):
    data[f1+ '+' +f2] = data[f1].astype(str) + '+'+ data[f2].astype(str)
    return (data)

In [74]:
make_conj(X_new, 'Pclass', 'Deck').head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Pclass+Deck
0,3,1,22.0,1,0,7.2500,2.0,NaN,3+nan
1,1,0,38.0,1,0,71.2833,0.0,C,1+C
2,3,0,26.0,0,0,7.9250,2.0,NaN,3+nan
3,1,0,35.0,1,0,53.1000,2.0,C,1+C
4,3,1,35.0,0,0,8.0500,2.0,NaN,3+nan


In [75]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(X_new.Pclass)
X_new['Pclass+Deck']=le.transform(X_new.Pclass)

In [76]:
del X_new['Deck']

In [77]:
X_test_new = X_test_new.join( X_test['Cabin'])

In [78]:
X_scaled_1 = preprocessing.scale(X_new)

In [79]:
X_test_new.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin
0,3,1,34.5,0,0,7.8292,1,NaN
1,3,0,47.0,1,0,7.0000,2,NaN
2,2,1,62.0,0,0,9.6875,1,NaN


In [80]:
X_test_new['Deck']=X_test_new['Cabin'].map(lambda x: substrings_in_string(str(x), cabin_list))
X_test_new.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin,Deck
0,3,1,34.5,0,0,7.8292,1,NaN,NaN
1,3,0,47.0,1,0,7.0000,2,NaN,NaN
2,2,1,62.0,0,0,9.6875,1,NaN,NaN


In [81]:
del X_test_new['Cabin']

In [82]:
make_conj(X_test_new, 'Pclass', 'Deck').head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Pclass+Deck
0,3,1,34.5,0,0,7.8292,1,NaN,3+nan
1,3,0,47.0,1,0,7.0000,2,NaN,3+nan
2,2,1,62.0,0,0,9.6875,1,NaN,2+nan
3,3,1,27.0,0,0,8.6625,2,NaN,3+nan
4,3,0,22.0,1,1,12.2875,2,NaN,3+nan


In [83]:
X_test_new['Pclass+Deck']=le.transform(X_test_new.Pclass)

In [84]:
X_test_new.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Pclass+Deck
0,3,1,34.5,0,0,7.8292,1,NaN,2
1,3,0,47.0,1,0,7.0000,2,NaN,2
2,2,1,62.0,0,0,9.6875,1,NaN,1


In [85]:
del X_test_new['Deck']

In [86]:
X_test_scaled_1 = preprocessing.scale(X_test_new)

In [88]:
from sklearn.neighbors import KNeighborsClassifier
array=[]
results=[]
k=1
while k<51:
    neigh = KNeighborsClassifier(n_neighbors=k)
    results = cross_val_score(neigh, X_scaled_1, y_train, cv=kf, scoring='accuracy')
   # print results.mean()
    array.append(results.mean())
    k = k+1
max_res = max(array)
print (max_res)
array.index(max_res)+1

0.8181972255351202


28

In [89]:
neigh2 = KNeighborsClassifier(n_neighbors=28)

In [90]:
neigh2.fit(X_scaled_1, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=28, p=2,
                     weights='uniform')

In [91]:
pred_8 = neigh2.predict(X_test_scaled_1)

In [92]:
d_knn8 = {"PassengerId":X_te['PassengerId'], "Survived": pd.Series(pred_8)}
df8 = pd.DataFrame(d_knn8)
df8.to_csv('kaggle_note_8', sep=',', index=False)

#score: 0.77033 - improved

In [93]:
X_new.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass+Deck
0,3,1,22.0,1,0,7.2500,2.0,2
1,1,0,38.0,1,0,71.2833,0.0,0
2,3,0,26.0,0,0,7.9250,2.0,2


Let's see how 'SibSp', 'Parch' fifluens on result

In [94]:
def make_conj_sibparch(data, f1, f2):
    data[f1+ '+' +f2] = data[f1]+ data[f2]
    return (data)

In [95]:
make_conj_sibparch(X_new, 'SibSp', 'Parch').head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass+Deck,SibSp+Parch
0,3,1,22.0,1,0,7.2500,2.0,2,1
1,1,0,38.0,1,0,71.2833,0.0,0,1
2,3,0,26.0,0,0,7.9250,2.0,2,0
3,1,0,35.0,1,0,53.1000,2.0,0,1
4,3,1,35.0,0,0,8.0500,2.0,2,0


In [96]:
make_conj_sibparch(X_test_new, 'SibSp', 'Parch').head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass+Deck,SibSp+Parch
0,3,1,34.5,0,0,7.8292,1,2,0
1,3,0,47.0,1,0,7.0000,2,2,1
2,2,1,62.0,0,0,9.6875,1,1,0
3,3,1,27.0,0,0,8.6625,2,2,0
4,3,0,22.0,1,1,12.2875,2,2,2


In [97]:
X_scaled_2 = preprocessing.scale(X_new)

In [98]:
X_test_scaled_2 = preprocessing.scale(X_test_new)

In [100]:
from sklearn.neighbors import KNeighborsClassifier
array=[]
results=[]
k=1
while k<51:
    neigh = KNeighborsClassifier(n_neighbors=k)
    results = cross_val_score(neigh, X_scaled_2, y_train, cv=kf, scoring='accuracy')
   # print results.mean()
    array.append(results.mean())
    k = k+1
max_res = max(array)
print (max_res)
array.index(max_res)+1

0.8170799070993662


14

In [101]:
neigh2 = KNeighborsClassifier(n_neighbors=14)
neigh2.fit(X_scaled_2, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=14, p=2,
                     weights='uniform')

In [102]:
pred_10 = neigh2.predict(X_test_scaled_2)

In [103]:
d_knn10 = {"PassengerId":X_te['PassengerId'], "Survived": pd.Series(pred_10)}
df10 = pd.DataFrame(d_knn10)
df10.to_csv('kaggle_note_10', sep=',', index=False)

#score: 0.77511 - improved